## Recommendations code:

### Imports:

In [96]:
import pandas as pd
import numpy as np

### Load in data and decode:

In [97]:
full = pd.read_csv('./data/strippeddata.csv')
df = pd.read_csv('./data/code_name_label.csv')
xy = pd.read_csv('./data/coords.csv')
rec1 = pd.read_csv('./data/recommender1.csv') # based on non-text aspects of the cache, + log sentiment


In [98]:
K0 = '0918273645.1425367.908.9012783465'

KEY = [pd.to_numeric(K0[7] +K0[0] +K0[24] +K0[20]+K0[10]+K0[8]), 
       pd.to_numeric(K0[13]+K0[20]+K0[0]  +K0[24]+K0[10]+K0[13]), 
       pd.to_numeric(K0[23]+K0[24]+K0[20] +K0[0] +K0[22]+K0[29])]

def CoordstoCode(xy_list):
    A = [np.power(xy,0.5) for xy in xy_list]
    c_list = [np.round(KEY[1]+aa) if aa%1>0.5 else np.round(KEY[1]-aa) for aa in A]
    c_list = [int(cc) for cc in c_list]
    return c_list

def OtherInfotoCode(z_list):
    A = [np.power(zz,0.5) for zz in z_list]
    c_list = [np.round(KEY[2]+aa) if aa%1>0.5 else np.round(KEY[2]-aa) for aa in A]
    c_list = [int(cc) for cc in c_list]
    return c_list

xy['code_index'] = CoordstoCode(list(xy['coords_index']))
xy = xy.sort_values(by='code_index')

df['Y'] = xy['latitude']
df['X'] = xy['longitude']

y_min = df['Y'].min()
y_max = df['Y'].max()
x_min = df['X'].min()
x_max = df['X'].max()

full = full[['difficulty', 'terrain', 'size', 'hints', 'is_premium', 'index']]
full['code_index'] = OtherInfotoCode(list(full['index']))
full.drop(columns=['index'],inplace=True)
full = full.sort_values(by='code_index')

ndexs = list(df.index)
codes = list(df['code']) 
code_index = {codes[n]:ndexs[n] for n in range(df.shape[0])}
index_code = {ndexs[n]:codes[n] for n in range(df.shape[0])}


### Functions:

In [99]:
def find_distance(code0,code_recs):
    x0 = list(df[df['code']==code0]['X'])[0]
    y0 = list(df[df['code']==code0]['Y'])[0]
    x = np.array([list(df[df['code']==c]['X'])[0] for c in code_recs]) - x0
    y = np.array([list(df[df['code']==c]['Y'])[0] for c in code_recs]) - y0
    x = (6371*np.cos(y0*np.pi/180)*np.pi/180)*x
    y = (6371*np.pi/180)*y
    distances = np.power((np.power(x,2) + np.power(y,2)),0.5)
    
    return distances

def find_clstr_recs(code):
    cluster = list(df[df['code']==code]['label'])[0]
    code_recs = list(df[df['label']==cluster]['code'])
    code_recs.remove(code)
    name_recs = list([list(df[df['code']==c]['name'])[0] for c in code_recs])
    dist_recs = find_distance(code,code_recs)
    
    N = len(code_recs)
    if N > 5:
        chk = pd.DataFrame({'code': code_recs, 'name': name_recs, 'distance': dist_recs})
        chk = chk.sort_values(by='distance')
        chk = chk[0:5]
        code_recs = list(chk['code'])
        name_recs = list(chk['name'])
        dist_recs  = list(chk['distance'])
  
    return code_recs, name_recs, dist_recs, N

def find_cos_recs(code):
    code_no = str(code_index[code])
    recs = list(rec1[[code_no]].sort_values(by=code_no).index)[1:6]    
    cdis_recs = list(rec1[[code_no]].sort_values(by=code_no)[code_no])[1:6]    
    code_recs = [df['code'][r] for r in recs]
    name_recs = [df['name'][r] for r in recs]
    dist_recs = find_distance(code,code_recs)
    
    return code_recs, name_recs, dist_recs, cdis_recs

def get_params(code_list):
    
    difficulties = []
    terrains     = []
    sizes        = []
    hints        = []
    premiums     = []
    for code in code_list:
        code_no = code_index[code]
        params = list(full.loc[code_no])
        difficulties.append(params[0])
        terrains.append(params[1])
        sizes.append(params[2])
        hints.append(params[3])
        premiums.append(params[4])
        
    return difficulties, terrains, sizes, hints, premiums

def get_kclusters(code,code_list):
    cluster0 = list(df[df['code']==code]['label'])[0]
    clusters = [1 if list(df[df['code']==c]['label'])[0]==cluster0 else 0 for c in code_list]
    return clusters

#### get user selected cache to start:

START User input: (1) enter code, (2) enter name (or partial), (3) enter latitude and longitude

(1) If user selects to enter code: have them enter it

(1A)    If the code is in the database, return code

(1B)    If not, prompt them to retype or start over entirely

(1B-i)          If they choose to retype, go back to (1)

(1B-ii)         If they choose to start over, go back to START

(2) If user selects to enter name: have them enter it

(2A)    Show the user the 4 names closest to what they typed (alphabetically), and allow them to choose one of them, retype, or choose start over entirely.
    
(2A-i)            If they choose a specific cache, return code
        
(2A-ii)           If they choose to retype, go back to (2)

(2A-iii)          If they choose to start over, go back to START

(3) If user selects to enter latitude & longitude: have them enter it

(3A)    Show the user the 4 names closest to what they typed (geographically), and allow them to choose one of them, retype, or choose start over entirely.
    
(3A-i)            If they choose a specific cache, return code
        
(3A-ii)           If they choose to retype, go back to (3)

(3A-iii)          If they choose to start over, go back to START





In [100]:
codes = list(df['code'])
names = list(df['name'])


if search_parameter=='Unknown':
    search_parameter = st.sidebar.radio('How would you like to choose a starting cache?', 
                                    ['By Geo Code (GXXXXXX)', 'By Name', 'By Coordinates'])

if search_parameter=='By Geo Code (GXXXXXX)':
    
    if code0=='Unknown':
        code0 = st.sidebar.text_input('Please input the code, starting with G:', value="GXXXXXX")
        if code0 in codes:
            code = code0
        else:
            retry1 = st.sidebar.radio('Sorry, that cache is not in the dataset.', 
                                    ['Try another code', 'Try a different way'])
            if retry1=='Try another code':
                code0 = 'Unknown'
            if retry1=='Try a different way':
                search_parameter = 'Unknown'
                
if search_parameter=='By Name':
    
    if name0=='Unknown':
        name0 = st.sidebar.text_input('Please type in what you can of the cache name:')
        if name0 in names:
            ndx = names.index(name0)
            code = codes[ndx]
        else:
            names1 = names.copy()
            names1.append(name0)
            names1.sort()
            n = names1.index(name0)
            n_list = list(range(5))
            if n>2: 
                n_list = [(j + n - 2) for j in n_list]
            if len(names1)-n < 3:
                n_list = [len(names1)+j for j in list(range(-5,0))]
            names4closest = [names1[j] for j in n_list]
            names4closest.remove(name0)
            options = names4closest
            options.append('Let me type it again')
            options.append('Try a different way')
            retry1 = st.sidebar.radio('Which cache did you mean?', options)
            if retry1 in names4closest:
                ndx = names.index(retry1)
                code = codes[ndx]
            if retry1=='Let me type it again':
                name0 = 'Unknown'
            if retry1=='Try a different way':
                search_parameter = 'Unknown'

if search_parameter=='By Coordinates':

    if lat0+lon0==0:
        lat0 = st.sidebar.text_input('Please input a decimal latitude value between 30 and 36 (north latitude)')
        lon0 = st.sidebar.text_input('Please input a decimal longitude value between -87 and -81 (west longitude is negative)')
        lats = np.array(df['Y'])
        lons = np.array(df['X'])
        dist = np.power((lats - lat0),2)
        dist +=np.power((lons - lon0)*np.cos(y0*np.pi/180),2)
        dist = np.power(dist,0.5)*6371*np.pi/180
        tempdf = pd.DataFrame({'ndex': list(df.index),'dist': dist, 'code': codes, 'name': name})       
        tempdf = tempdf.sort_values(by='dist')
        tempdf = tempdf[0:4]
        codes1 = list(tempdf['code'])
        names1 = list(tempdf['name'])
        options1 = [codes1[j]+': '+names1[j] for j in range(4)]
        options.append('Let me type it again')
        options.append('Try a different way')
        retry1 = st.sidebar.radio('Which cache did you mean?', options) 
        if retry1 in options1:
            code = codes1.index(retry1)
        if retry1=='Let me type it again':
            lat0 = 0
            lon0 = 0
        if retry1=='Try a different way':
            search_parameter = 'Unknown'
        


NameError: name 'search_parameter' is not defined

In [ ]:

def get_outpt(code):

    # K means recommendations (0-5):
    code_recsK, name_recsK, dist_recsK, N = find_clstr_recs(code)
    diff_recsK, terr_recsK, size_recsK, hint_recsK, prem_recsK = get_params(code_recsK)
    dist_recsK = list(dist_recsK)

    if N==0:
        outpt = 'You picked a somewhat unusual cache - there are no other caches in our 12000+ within the same KMeans cluster.'
    elif N<=5:
        outpt = 'Here are all of the caches within the same KMeans cluster:'
        outpt+='\n'
        for n in range(N):
            outpt +='\n' + code_recsK[n] + ': ' + name_recsK[n]
    else:         
        outpt = 'There were many caches within the same KMeans cluster, so here are the 5 that are geographically closest:'
        outpt+='\n'
        for n in range(5):
            outpt +='\n' + code_recsK[n] + ': ' + name_recsK[n]
    if N>0:
        outpt+='\n'
        if np.max(diff_recsK)<=3:   outpt+='\nThese are all nice and easy '
        elif np.min(diff_recsK)<=3: outpt+='\nSome of these are higher difficulty '
        else:                       outpt+='\nBetter put on your thinking cap for these tough caches '
        if np.max(terr_recsK)<=3:   outpt+='and the terrain should be not too bad.'
        elif np.min(terr_recsK)<=3: outpt+='and some have some rough terrain.'
        else:                       outpt+='and hiking boots and TecNu may be in order for these.'


    # cos dist recommendations (5):
    code_recs1, name_recs1, dist_recs1, pdis_recs1 = find_cos_recs(code)
    cltr_recs1 = get_kclusters(code,code_recs1)
    diff_recs1, terr_recs1, size_recs1, hint_recs1, prem_recs1 = get_params(code_recs1)
    dist_recs1 = list(dist_recs1)

    outpt+='\n\n'
    outpt+= 'Here are the five caches with the lowest Cosine Distance from your cache. Note that cos dist is always between 0 and 2.'
    outpt+='\n'
    for n in range(5):
        outpt +='\n' + code_recs1[n] + ': ' + name_recs1[n] + ', cos dist = ' + str(np.round(pdis_recs1[n],3))
        if cltr_recs1[n]==1: outpt+= ', also in the same KMeans cluster'

    outpt+='\n'
    if np.max(diff_recs1)<=3:   outpt+='\nThese are all nice and easy '
    elif np.min(diff_recs1)<=3: outpt+='\nSome of these are higher difficulty '
    else:                       outpt+='\nBetter put on your thinking cap for these tough caches '
    if np.max(terr_recs1)<=3:   outpt+='and the terrain should be not too bad.'
    elif np.min(terr_recs1)<=3: outpt+='and some have some rough terrain.'
    else:                       outpt+='and hiking boots and TecNu may be in order for these.'

    outpt+='\n\n'
    outpt+='Thank you, and have fun! Email me at clh@cholland.me if you have any comments or feedback.'


    #print(outpt)
    return outpt


In [101]:
code = 'GC5GECD'
outpt = get_outpt(code)

In [103]:
print(outpt)

There were many caches within the same KMeans cluster, so here are the 5 that are geographically closest:

GC5GDM3: Holy Cow!  Not Another One!
GC5GEDD: The Blues #126 - Deal The Cards
GC5GEHH: Grilled Cheese
GC3MQQH: Robbie's Evil Lock-N-Lock Puzzle
GC5GA2E: E’s First Cache

Some of these are higher difficulty and some have some rough terrain.

Here are the five caches with the lowest Cosine Distance from your cache. Note that cos dist is always between 0 and 2.

GC5GECD:  "Let's Play Ball!" #1, cos dist = 0.0, also in the same KMeans cluster
GC5GDB8: Log it., cos dist = 0.0, also in the same KMeans cluster
GC5GD96: The Blues #162 - Black Cat Bone, cos dist = 0.0, also in the same KMeans cluster
GC5GEAY: GA400 Overpasses of FoCo - Majors Road, cos dist = 0.0, also in the same KMeans cluster
GC5GD8J: Gristmill, cos dist = 0.0, also in the same KMeans cluster

These are all nice and easy and the terrain should be not too bad.

Thank you, and have fun! Email me at clh@cholland.me if you 

In [38]:
ndx_list = [10, 40, 55, 540, 1000, 4958, 5002]
codes = [df.loc[n]['code'] for n in ndx_list]

for code in codes:
    print(code)
    print('----------')
    print()
    outpt = get_outpt(code)

GC8N2JF
----------

There were many caches within the same KMeans cluster, so here are the 5 that are geographically closest:

GC90JDX: Milledge Mystery Renewed
GC4NDQP: Color Code: Hello, Yellow!
GC8ZQT3: "Casa de Waffle #9"
GC44MZ3: Snoopy's Library
GCN560: Close to Home Multi

These are all nice and easy and the terrain should be not too bad.

Here are the five caches with the lowest Cosine Distance from your cache. Note that cos dist is always between 0 and 2.

GC90A8J: I Scream, You Scream, We All Scream..., cos dist = 0.025, also in the same KMeans cluster
GC98ZVQ: Blue Bird, cos dist = 0.043, also in the same KMeans cluster
GC98ZW4: #173 Baker County, cos dist = 0.048, also in the same KMeans cluster
GC9A68X: Happy Anniversary Year 1, cos dist = 0.05, also in the same KMeans cluster
GC99RDV: McGregor's Thief, cos dist = 0.059

These are all nice and easy and the terrain should be not too bad.

Thank you, and have fun! Email me at clh@cholland.me if you have any comments or feedb

In [89]:
# Get output for every cache, and save to disk, for faster app: run this in makerecommender.py on the command line, it takes a while.

In [ ]:
codes = list(df['code'])

output_strings = []
for n,code in enumerate(codes):
    outpt = get_outpt(code)
    output_strings.append(outpt)
    print(f'Cache #{n}: {code}, {n*100/12432}% completed ...')   
    
df['output string'] = output_strings
df.to_csv('./data/recommendations_file.csv',index=False)
